# Renat Rezyapov

## Which player statistics have the largest effect on their position in the ranking based on points?

In order to answer this question, the change in ranking between the two consecutive seasons for players will serve as a point of comparisson with statistics in the earlier year. 
The purpose of the question is to bring attention to which parameters the players can focus on imrproving in order to advance in the ranking based on the number of points in the next season. 

In [1]:
import pandas as pd
nhl2017 = pd.read_csv('../data/processed/Data301ProjectData2017.csv')
nhl2018 = pd.read_csv('../data/processed/Data301ProjectData2018.csv')